In [138]:
import requests
import json
import os
import foursquare
import pandas as pd
from tqdm import tqdm
import time
from time import sleep

# Foursquare

### Send a request to Foursquare with a small radius (1000m) for all the bike stations in your city of choice. 

#### Step 1: Set code to review full API

In [129]:
# Setup Foursquare API credentials 
api_key = os.environ.get('FOURSQUARE_API_KEY')

# Define the endpoint URL for the Foursqare request
endpoint = "https://api.foursquare.com/v3/places/search"

# Define the query parameters to include only the latitude and longitude fields
def foursquare_query(latitude, longitude):
    params = {
        'll': f'{latitude},{longitude}', 'radius':'1000', 'query':'Restaurant,  Landmarks and Outdoors > Park'
        }
    #Headers specific to FourSquare - used to specify wanted metadata - not typically found under all APIs. Found under https://location.foursquare.com/developer/reference/authentication
    headers = {"Accept": "application/json",
        "Authorization": api_key
        }

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content



#### Step 2: Retrieve Foursquare results

In [130]:
#Retrieve the Foursquare query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = foursquare_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Foursquare query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, location_name):
    json_content = foursquare_query(latitude, longitude)
    poi_list = []
    for poi in json_content['results']:
        poi_dict = {}
        poi_dict['fsq_id'] = poi['fsq_id']
        poi_dict['location_name'] = location_name
        poi_dict['poi_type'] = [category['name'] for category in poi['categories']]
        poi_list.append(poi_dict)
    return poi_list

#### Step 3: Parse through data to retrieve point of interest (POI) details (Available from Foursquare - location, type)

In [139]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')

rand_df = bike_df.sample(n=250)

# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()


# create an empty list to store the dictionaries
data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    fsq_ids = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for fsq_id in fsq_ids:
        data_list.append({'fsq_id': fsq_id['fsq_id'], 'location_name': fsq_id['location_name'], 'poi_type': fsq_id['poi_type']})
    time.sleep(1) # add 1 second delay between each iteration

100%|██████████| 250/250 [05:59<00:00,  1.44s/it]


#### Step 4:  Put your parsed results into a DataFrame

In [140]:
foursquare_poi_df = pd.DataFrame(data_list)

# Yelp

### Send a request to Yelp with a small radius (1000m) for all the bike stations in your city of choice. 

##### Set code to review full API for restaurants on Yelp

In [143]:
# Setup Yelp API credentials 
api_key = os.environ.get('YELP_API_KEY')

# Define the endpoint URL for the Yelp request
endpoint = "https://api.yelp.com/v3/businesses/search"

# Define the query parameters to include only the latitude and longitude fields
def yelp_query(latitude, longitude):
    params = {
        'latitude': latitude , 'longitude': longitude, 'radius': 1000, 'term': 'restaurant'
        }
    #Headers specific to Yelp - used to specify wanted metadata - not typically found under all APIs.
    headers = {'Authorization': f'Bearer {api_key}'}

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content

##### Parse through the response to get the POI on restaurants to retrieve details on restaurant name, location, rating, price, and classifacation

In [144]:
#Retrieve the Yelp query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = yelp_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Yelp query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, name):
    json_content = yelp_query(latitude, longitude)
    poi_list = []
    for poi in json_content['businesses']:
        poi_dict = {}
        poi_dict['id'] = poi['id']
        poi_dict['location'] = name
        poi_dict['categories'] = poi['categories']
        poi_dict['rating'] = poi['rating']
        poi_dict['name'] = poi['name']
        try:
            poi_dict['rating'] = poi['rating']
        except:
            poi_dict['rating'] = ''
        try:
            poi_dict['price'] = poi['price']
        except:
            poi_dict['price'] = ''
        poi_list.append(poi_dict)
    return poi_list

#### Run loop through CityBikes CSV file

In [145]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')


# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()


# create an empty list to store the dictionaries
yelp_restaurant_data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    id = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for id in id:
        yelp_restaurant_data_list.append({'id': id['id'], 'name': id['name'], 'location': id['location'], 'categories': id['categories'], 'rating': id['rating'],'price': id['price']})
        

100%|██████████| 250/250 [02:51<00:00,  1.45it/s]


##### Put your parsed results into a DataFrame

In [146]:
yelp_restaurants_df = pd.DataFrame(yelp_restaurant_data_list)

## Yelp Parks

##### Set code to review full API for parks on Yelp

In [147]:
# Setup Yelp API credentials 
api_key = os.environ.get('YELP_API_KEY')

# Define the endpoint URL for the Yelp request
endpoint = "https://api.yelp.com/v3/businesses/search"

# Define the query parameters to include only the latitude and longitude fields
def yelp_query(latitude, longitude):
    params = {
        'latitude': latitude , 'longitude': longitude, 'radius': 1000, 'term': 'parks'
        }
    #Headers specific to Yelp - used to specify wanted metadata - not typically found under all APIs.
    headers = {'Authorization': f'Bearer {api_key}'}

    # Send an HTTP GET request to the endpoint URL with the query parameters
    response = requests.get(endpoint, params=params, headers=headers)

    # Get the JSON content from the response
    json_content = response.json()

    # Print out the entire JSON response to see catergory names
    return json_content

##### Parse through the response to get the POI on parks to retrieve details on park name, location, rating, any associated price, and classifacation.

In [148]:
#Retrieve the Yelp query results for a given latitude and longitude
def bike_radius(latitude, longitude):
    json_content = yelp_query(latitude, longitude)
    poi = json_content['results']
    return json_content

#extract the relevant information from the Yelp query result and store it in a dictionary that is returned as a list.
def bike_radius_info(latitude, longitude, name):
    json_content = yelp_query(latitude, longitude)
    poi_list = []
    for poi in json_content['businesses']:
        poi_dict = {}
        poi_dict['id'] = poi['id']
        poi_dict['location'] = name
        poi_dict['categories'] = poi['categories']
        poi_dict['rating'] = poi['rating']
        poi_dict['name'] = poi['name']
        try:
            poi_dict['rating'] = poi['rating']
        except:
            poi_dict['rating'] = ''
        try:
            poi_dict['price'] = poi['price']
        except:
            poi_dict['price'] = ''
        poi_list.append(poi_dict)
    return poi_list

#### Run loop through CityBikes CSV file

In [149]:
# read the file containing the locations
bike_df = pd.read_csv('/Users/brittanyharding/LHL-Projects/Statistical-Modelling-with-Python/notebooks/London_CityBikes_Data.csv')


# extract the name, latitude, and longitude columns from the City Bike Data
names = rand_df['name'].unique()
latitudes = rand_df['latitude'].unique()
longitudes = rand_df['longitude'].unique()


# create an empty list to store the dictionaries
yelp_park_data_list = []

# loop through each location and call the bike_radius_info function
for i in tqdm(range(len(latitudes))):
    id = bike_radius_info(latitudes[i], longitudes[i], names[i])
    for id in id:
        yelp_park_data_list.append({'id': id['id'], 'name': id['name'], 'location': id['location'], 'categories': id['categories'], 'rating': id['rating'],'price': id['price']})
        

100%|██████████| 250/250 [02:16<00:00,  1.83it/s]


#### Step 4:  Put your parsed results into a DataFrame

In [150]:
yelp_parks_df = pd.DataFrame(yelp_park_data_list)

# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating